# Generate Code Using the Big Code Bench Dataset
This experiment is done using:
* 10 small language models (SLMs)
* 3 types of prompt (no prompt, some instructions, detailed instructions)
* 3 SLM temperature settings
* 3 SLM top_p settings

In [1]:
# mount Google Drive
from google.colab import drive

drive.mount('/content/drive')             # Mount Google Drive to Colab session
!ls /content/drive/MyDrive                # Verify that your drive is accessible

Mounted at /content/drive
'16 б.gdoc'				    Labwork_Andrew_Nedilko_20250219.pdf
'6 фото.     5 лет блондин.JPG'		   'Most Asked ML Questions.pdf'
 Apple					    rental_houses.gsheet
 Archive.zip				    share
'Colab Notebooks'			   'Tanya Passport.JPG'
 futuristic-office.jpeg			   'лечение травами 3.gdoc'
'Labwork_Andrew_Nedilko_20250219 (1).pdf'


In [2]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/')

from prompts import ( complete_code_prompt_basic, complete_code_prompt, complete_code_prompt_full,
                      complete_task_prompt_basic, complete_task_prompt, complete_task_prompt_full, )
from helpers import get_tokenizer, get_model, generate_response, clean_code, write_jsonl, read_problems

In [3]:
import logging
import sys
import time, datetime
import os
import torch
from google.colab import userdata

# get HuggingFace token from secrets
HF_TOKEN = userdata.get('HF_TOKEN')       # Retrieve token from Colab Secrets
os.environ['HF_TOKEN'] = HF_TOKEN         # Set token as env variable (recommended optional)
from huggingface_hub import HfFolder
HfFolder.save_token(HF_TOKEN)             # Store the token locally to avoid re-authentication

In [4]:
# select model, propmt, temperature, top_p
models = [
    ('mlabonne/phixtral-2x2_8', 'phixtral-2x2_8'),
    ('mlabonne/phixtral-4x2_8', 'phixtral-4x2_8'),
    ('NousResearch/Nous-Hermes-2-SOLAR-10.7B', 'Nous-Hermes-2-Solar-10.7B'),
    ('meta-llama/Meta-Llama-3.1-8B-Instruct', 'Meta-Llama-3.1-8B-Instruct'),
    ('google/codegemma-7b-it', 'codegemma-7b-it'),
    ('deepseek-ai/deepseek-coder-6.7b-instruct', 'deepseek-coder-6.7b-instruct'),
    ('m-a-p/OpenCodeInterpreter-DS-6.7B', 'OpenCodeInterpreter-DS-6.7B'),
    ('Artigenz/Artigenz-Coder-DS-6.7B', 'Artigenz-Coder-DS-6.7B'),
    ('Qwen/CodeQwen1.5-7B-Chat', 'CodeQwen1.5-7B-Chat'),
    ('NTQAI/Nxcode-CQ-7B-orpo', 'Nxcode-CQ-7B-orpo'), ]
# actual prompt & prompt name for log file name
prompts_and_names = [
    (complete_code_prompt_basic, 'complete_code_prompt_basic'),
    (complete_code_prompt,       'complete_code_prompt'),
    (complete_code_prompt_full,  'complete_code_prompt_full'),
    (complete_task_prompt_basic, 'complete_task_prompt_basic'),
    (complete_task_prompt,       'complete_task_prompt'),
    (complete_task_prompt_full,  'complete_task_prompt_full'), ]
# temperature and top_k values and their labels for file names
temperature_values = [
    (1.0,  'temperature1.0'),
    (0.75, 'temperature0.75'),
    (0.5,  'temperature0.5'),
    (0.005, 'temperature0.005'),]
top_p_values = [
    (1.0,  'topP1.0'),
    (0.75, 'topP0.75'),
    (0.5,  'topP0.5'),]

model_idx       = 8
prompt_idx      = 3
temperature_idx = 2
top_p_idx       = 0

model_name, model_nickname     = models[ model_idx ]
my_prompt, my_prompt_label     = prompts_and_names[ prompt_idx ]
TEMPERATURE, temperature_label = temperature_values[ temperature_idx ]
TOP_P, top_p_label             = top_p_values[ top_p_idx ]

SPECIAL_MESSAGE = f'Model temperature: {TEMPERATURE}. Model top_p: {TOP_P}. Model top_k: 50. Model dtype: torch.float32'

print(f'Model name:     {model_name}')
print(f'Model nickname: {model_nickname}')
print(f'Prompt name:    {my_prompt_label}')
print(f'Prompt:\n{my_prompt.strip()}\n')
print(f'Temperature: {TEMPERATURE}. Temperature label: {temperature_label}')
print(f'Top p: {TOP_P}. Top p label: {top_p_label}')
print(f'Special message: {SPECIAL_MESSAGE}')

Model name:     Qwen/CodeQwen1.5-7B-Chat
Model nickname: CodeQwen1.5-7B-Chat
Prompt name:    complete_task_prompt_basic
Prompt:
Complete the following task:
{}.

Your output code must satisfy these tests:
{}

Temperature: 0.5. Temperature label: temperature0.5
Top p: 1.0. Top p label: topP1.0
Special message: Model temperature: 0.5. Model top_p: 1.0. Model top_k: 50. Model dtype: torch.float32


In [5]:
# save results
time_stamp     = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-2]
results_file   = f'/content/drive/My Drive/Colab Notebooks/logs/{model_nickname}_{my_prompt_label}_{temperature_label}_{top_p_label}_completions_{time_stamp}.jsonl'
log_file       = f'/content/drive/My Drive/Colab Notebooks/logs/logs/{model_nickname}_{my_prompt_label}_{temperature_label}_{top_p_label}_log_{time_stamp}.log'

# log results
for handler in logging.root.handlers[:]:            # overwrite any previous handlers with different formats
    logging.root.removeHandler(handler)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s  %(levelname)s  %(message)s',
    handlers=[
        logging.FileHandler(log_file),              # Log to a file
        logging.StreamHandler(sys.stdout)           # Log to console (default - sys.stderr (red background)
    ]
)

# print job config
logging.info('*****     JOB CONFIGURATION     *****')
logging.info(f'MODEL: {model_name}')
logging.info(f'MODEL NICKNAME: {model_nickname}')
logging.info(SPECIAL_MESSAGE)
logging.info(f'GENERATED CODE SAVED IN: {results_file}')
logging.info(f'THIS LOG FILE: {log_file}')
if my_prompt in [complete_code_prompt_basic, complete_code_prompt, complete_code_prompt_full,]:
  logging.info(f'PROMPT:\n{my_prompt.lstrip().format("Starter Code")}')
else:
  logging.info(f'\nPROMPT:\n{my_prompt.format("Task Description", "Test Cases")}')
logging.info('*****     END OF JOB CONFIGURATION     *****')

2025-04-24 10:30:41,431  INFO  *****     JOB CONFIGURATION     *****
2025-04-24 10:30:41,433  INFO  MODEL: Qwen/CodeQwen1.5-7B-Chat
2025-04-24 10:30:41,433  INFO  MODEL NICKNAME: CodeQwen1.5-7B-Chat
2025-04-24 10:30:41,434  INFO  Model temperature: 0.5. Model top_p: 1.0. Model top_k: 50. Model dtype: torch.float32
2025-04-24 10:30:41,435  INFO  GENERATED CODE SAVED IN: /content/drive/My Drive/Colab Notebooks/logs/CodeQwen1.5-7B-Chat_complete_task_prompt_basic_temperature0.5_topP1.0_completions_20250424_103041_0909.jsonl
2025-04-24 10:30:41,436  INFO  THIS LOG FILE: /content/drive/My Drive/Colab Notebooks/logs/logs/CodeQwen1.5-7B-Chat_complete_task_prompt_basic_temperature0.5_topP1.0_log_20250424_103041_0909.log
2025-04-24 10:30:41,436  INFO  
PROMPT:

Complete the following task:
Task Description.

Your output code must satisfy these tests:
Test Cases

2025-04-24 10:30:41,437  INFO  *****     END OF JOB CONFIGURATION     *****


In [6]:
# read dataset
file = '/content/drive/My Drive/Colab Notebooks/data/LBPP_Test.jsonl.gz'
tasks = read_problems(file)
print(f'Type of tasks: {type(tasks)}\n')

counter = 0
for k,v in tasks.items():
    if counter == 3:
        break
    print(f'task_id: {k}', type(k))
    for k2, v2 in v.items():
        print(f'\n{k2}:\n{v2}')
    print('\n' + '='*100 + '\n')
    counter += 1

Type of tasks: <class 'dict'>

task_id: lbpp/0 <class 'str'>

task_id:
lbpp/0

test:
import numpy as np

ar = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
expected_output = np.array([[1, 2, 3, 2.0, 0.81649658], [4, 5, 6, 5.0, 0.81649658], [7, 8, 9, 8.0, 0.81649658]])
assert np.allclose(add_avg_and_std_cols_numpy(ar), expected_output)
ar = np.array([[1], [1], [2]])
expected_output = np.array([[1, 1, 0.0], [1, 1, 0.0], [2, 2, 0.0]])
assert np.allclose(add_avg_and_std_cols_numpy(ar), expected_output)
ar = np.array([[1, 2, 3, 4, 5]])
expected_output = np.array([[1, 2, 3, 4, 5, 3.0, 1.41421356]])
assert np.allclose(add_avg_and_std_cols_numpy(ar), expected_output)

prompt:
Write a python function `add_avg_and_std_cols_numpy(ar: np.ndarray) -> np.ndarray` that takes a 2D numpy array and returns a 2D numpy array with two additional columns appended to the end. The first column should contain the average of each row, and the second column should contain the standard deviation of each row. You ma

In [7]:
torch_dtype = torch.float32
#torch_dtype = torch.bfloat16
tokenizer = get_tokenizer(model_name)
model     = get_model(model_name, torch_dtype=torch_dtype)

tokenizer_config.json:   0%|          | 0.00/972 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.46M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/1.42M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/31.7k [00:00<?, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/3.89G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/2.71G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

In [8]:
print(f'Model device: {model.device}')
print(f'Model temperature: {model.config.temperature}. Model top_p: {model.config.top_p}. Model top_k: {model.config.top_k}. Model dtype: {model.dtype}')

input_prompt = "What is the capital of California?"
print(input_prompt)
generate_response(input_prompt, tokenizer, model, temperature=TEMPERATURE)

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Model device: cuda:0
Model temperature: 1.0. Model top_p: 1.0. Model top_k: 50. Model dtype: torch.float32
What is the capital of California?


'The capital of California is Sacramento.'

In [ ]:
num_samples_per_task = 1
completions  = []

for task_id, task_body in tasks.items():
    logging.info( task_id )
    #if task_id == 'lbpp/5':
    #        break
    for i in range(num_samples_per_task):
        start_time  = time.time()
        full_prompt = my_prompt.lstrip().format( task_body['prompt'], task_body['test'] )
        try:
            completion = generate_response( full_prompt, tokenizer, model, temperature=TEMPERATURE )
        except Exception as e:
            completion = f"Error generating a completion:\n{e}"

        completions.append( {'task_id': task_id, 'completion': completion} )
        write_jsonl(results_file, completions)
        logging.info('PROMPT:\n' + full_prompt + '\n')
        logging.info('COMPLETION:\n' + completion + '\n')
        logging.info(f"Time elapsed: {(time.time() - start_time):.4f} seconds\n")

2025-04-24 10:32:02,951  INFO  lbpp/0
2025-04-24 10:32:22,646  INFO  PROMPT:
Complete the following task:
Write a python function `add_avg_and_std_cols_numpy(ar: np.ndarray) -> np.ndarray` that takes a 2D numpy array and returns a 2D numpy array with two additional columns appended to the end. The first column should contain the average of each row, and the second column should contain the standard deviation of each row. You may assume that the input array has at least one row and one column..

Your output code must satisfy these tests:
import numpy as np

ar = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
expected_output = np.array([[1, 2, 3, 2.0, 0.81649658], [4, 5, 6, 5.0, 0.81649658], [7, 8, 9, 8.0, 0.81649658]])
assert np.allclose(add_avg_and_std_cols_numpy(ar), expected_output)
ar = np.array([[1], [1], [2]])
expected_output = np.array([[1, 1, 0.0], [1, 1, 0.0], [2, 2, 0.0]])
assert np.allclose(add_avg_and_std_cols_numpy(ar), expected_output)
ar = np.array([[1, 2, 3, 4, 5]])
expecte

In [ ]:
from google.colab import runtime
runtime.unassign()